In [ ]:
# Import and create the handler
from json_handler import JSONHandler
handler = JSONHandler("./Reduced/losses_log_reduced.json")

# Read the file
dictionary_list = handler.read_json_file()
print(f"The file has the following number of elements:\n\t{len(dictionary_list)}")

In [ ]:
# Create a list where to store all the dictionaries for the final plotting
final_dict_list = []
# Create two temporary variables where to store the losses of the training
temp_loss = 0
temp_counter = 0
# For cycle for the processing of data
for i in range(len(dictionary_list)):
    # When the element is of type "training", update the temporary variables (data is cumulative)
    if dictionary_list[i]['type'] == "training":
        # Else, just update the temporary variables
        temp_loss += dictionary_list[i]['total_loss']
        temp_counter += 1
    # When the element is "validation", finalize the storage of training data and store the validation data
    else:
        final_dict_list.append(
            {'type': "training",
            'loss': (temp_loss/temp_counter)}
        )
        temp_loss=0
        temp_counter=0
        final_dict_list.append(
            {'type': "validation",
             'loss': dictionary_list[i]['total_loss'],
             'ap40': dictionary_list[i]['ap40']}
        )



for element in final_dict_list:
    print(element)

In [ ]:
import matplotlib.pyplot as plt
import numpy

# Define the validation interval
val_interval = 5

# Separate the data into training and validation lists
training_losses = [entry['loss'] for entry in final_dict_list if entry['type'] == 'training']
validation_losses = [entry['loss'] for entry in final_dict_list if entry['type'] == 'validation']
metric_ap40 = [entry['ap40']*100 for entry in final_dict_list if entry['type'] == 'validation']

# Create a range of indices for the X axis
epochs = numpy.linspace(val_interval, (len(training_losses))*val_interval, len(training_losses))

# Plot the training and validation losses
plt.figure(figsize=(15, 10))
plt.plot(epochs, training_losses, 'b-', marker='o', label='Training Loss')  # Blue line for training
plt.plot(epochs, validation_losses, 'r-', marker='o', label='Validation Loss')  # Red line for validation
plt.plot(epochs, metric_ap40, 'k-', marker='o', label='Metric: 3D AP40')  # Black line for ap40

# Set axes to start from 0
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.grid(True)

# Add labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')

# Add annotations for each point (training)
for i, loss in enumerate(training_losses):
    plt.annotate(f'{loss:.2f}', (epochs[i], training_losses[i]), textcoords="offset points", xytext=(0,7), ha='center', fontsize=8, color='blue')

# Add annotations for each point (validation)
for i, loss in enumerate(validation_losses):
    plt.annotate(f'{loss:.2f}', (epochs[i], validation_losses[i]), textcoords="offset points", xytext=(0,7), ha='center', fontsize=8, color='red')

# Add annotations for each point (validation)
for i, ap40 in enumerate(metric_ap40):
    plt.annotate(f'{ap40:.2f}', (epochs[i], metric_ap40[i]), textcoords="offset points", xytext=(0,7), ha='center', fontsize=8, color='black')

# Show the legend
plt.legend()

# Display the plot
plt.show()